In [10]:
import numpy as np

def sorting_eig(s, u):
    order = np.argsort(s)
    s_res, u_res = s.copy(), u.copy()
    for i in range(len(order)):
        s_res[i] = s[order[i]]
        u_res[i] = u[order[i]]
    return s_res, u_res

def svd(a):
    if a.shape[0] >= a.shape[1]:
        m = a @ a.T
        s, u = np.linalg.eig(m)
        s, u = sorting_eig(s, u)
        s = np.diagflat(np.append(s, [0]))
        for i in range(len(s)):
            if (s[i][i] == 0):
                s = s[:i, :i]
                u = u[:, :i]
        s_inv = s.copy()
        for i in range(len(s)):
            s_inv[i][i] = s_inv[i][i] ** (-1)
        v = s_inv @ u.T @ a
        return u, s, v

    m = a.T @ a
    s, v = np.linalg.eig(m)
    s = np.diagflat(np.append(s, [0]))
    for i in range(len(s)):
        if (s[i][i] == 0):
            s = s[:i, :i]
            v = v[:i,:]
    s_inv = s.copy()
    for i in range(len(s)):
        s_inv[i][i] = s_inv[i][i] ** (-1)
    u = s_inv @ v.T @ a
    return u, s, v

In [11]:
u, s, v = svd(np.array([[1, 1, 0],
                        [1, 1, 0],
                        [0, 0, 1],
                        [1, 0, -1]]))
print(u @ s @ v, sep = '\n')

[[  1.00000000e+00   1.00000000e+00   7.74116373e-16]
 [  1.00000000e+00   1.00000000e+00   9.75476693e-16]
 [  8.32667268e-16   6.38378239e-16   1.00000000e+00]
 [  1.00000000e+00  -1.11022302e-15  -1.00000000e+00]]
